In [1]:
import torch
from torch import nn
from torch.nn import functional as F

In [3]:
net = nn.Sequential(nn.Linear(20,256), nn.ReLU(), nn.Linear(256,4))

x = torch.rand(4,20)

net(x)

tensor([[ 0.0783,  0.3590, -0.0731, -0.0467],
        [-0.1070,  0.3206,  0.0246,  0.0358],
        [ 0.0253,  0.3518, -0.0397, -0.1480],
        [ 0.0657,  0.2663,  0.0237, -0.1640]], grad_fn=<AddmmBackward0>)

In [19]:
class MLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear = nn.Linear(20,256)
        self.out = nn.Linear(256,4)
        
    def forward(self, x):
        x = self.linear(x)
        return self.out(x)

In [20]:
net_2= MLP()
net_2(x)

tensor([[ 0.1203, -0.2039, -0.2938, -0.2191],
        [-0.2447, -0.0979, -0.0021, -0.4016],
        [-0.1687, -0.1711, -0.3016, -0.2075],
        [-0.1183, -0.2126, -0.0921, -0.2839]], grad_fn=<AddmmBackward0>)

In [22]:
class MySequential(nn.Module):
    def __init__(self, *args):
        super().__init__()
        for block in args:
            self._modules[block] = block

    def forward(self, X):
        for block in self._modules.values():
            X = block(X)
        return X

net = MySequential(nn.Linear(20, 256), nn.ReLU(), nn.Linear(256, 10))
net(x)

tensor([[-0.1391, -0.0278,  0.2070, -0.1334, -0.1563,  0.2678,  0.0795,  0.0257,
          0.2009,  0.0765],
        [-0.1829, -0.0499,  0.1507, -0.0139, -0.2331,  0.1248,  0.0177,  0.0632,
          0.1014,  0.0550],
        [-0.1354, -0.0252,  0.1293, -0.0891, -0.2295,  0.1582,  0.0376, -0.0202,
          0.0172, -0.0294],
        [-0.2174, -0.1077,  0.2495, -0.0039, -0.2336,  0.1664,  0.0833, -0.0337,
          0.1192,  0.0973]], grad_fn=<AddmmBackward0>)

# 参数管理

In [23]:
import torch
from torch import nn

net = nn.Sequential(nn.Linear(4, 8), nn.ReLU(), nn.Linear(8, 1))
X = torch.rand(size=(2, 4))
net(X)

tensor([[0.0440],
        [0.1434]], grad_fn=<AddmmBackward0>)

In [24]:
net.state_dict()

OrderedDict([('0.weight',
              tensor([[-0.3504,  0.4010, -0.1288,  0.0515],
                      [-0.0426, -0.1163,  0.1041,  0.3822],
                      [ 0.2477,  0.3935, -0.2827, -0.0198],
                      [-0.4825,  0.1456, -0.3439, -0.3355],
                      [ 0.4154,  0.0788, -0.4797,  0.1925],
                      [ 0.0071, -0.4424,  0.0831,  0.3184],
                      [ 0.1483, -0.4746, -0.1701, -0.1730],
                      [-0.3335, -0.4982,  0.1955,  0.1257]])),
             ('0.bias',
              tensor([-0.0929,  0.3424,  0.3592, -0.0089, -0.0341, -0.2864,  0.0128,  0.4610])),
             ('2.weight',
              tensor([[ 0.1192, -0.2266, -0.2769, -0.3365,  0.0446,  0.3057, -0.0140, -0.0112]])),
             ('2.bias', tensor([0.3179]))])

In [25]:
net[2].bias

Parameter containing:
tensor([0.3179], requires_grad=True)

In [26]:
net[2].bias.data

tensor([0.3179])

In [28]:
type(net[2].bias)

torch.nn.parameter.Parameter

In [31]:
net.named_parameters

<bound method Module.named_parameters of Sequential(
  (0): Linear(in_features=4, out_features=8, bias=True)
  (1): ReLU()
  (2): Linear(in_features=8, out_features=1, bias=True)
)>

In [32]:
def block1():
    return nn.Sequential(nn.Linear(4, 8), nn.ReLU(), nn.Linear(8, 4),
                         nn.ReLU())

def block2():
    net = nn.Sequential()
    for i in range(4):
        net.add_module(f'block {i}', block1())
    return net

rgnet = nn.Sequential(block2(), nn.Linear(4, 1))
rgnet(X)

tensor([[-0.3059],
        [-0.3059]], grad_fn=<AddmmBackward0>)

In [33]:
rgnet

Sequential(
  (0): Sequential(
    (block 0): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block 1): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block 2): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block 3): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
  )
  (1): Linear(in_features=4, out_features=1, bias=True)
)

In [34]:
def init_normal(m):
    if type(m) == nn.Linear:
        nn.init.normal_(m.weight, mean=0, std=0.01)
        nn.init.zeros_(m.bias)

net.apply(init_normal)
net[0].weight.data[0], net[0].bias.data[0]

(tensor([-0.0335,  0.0018,  0.0103, -0.0139]), tensor(0.))

In [35]:
shared = nn.Linear(8, 8)
net = nn.Sequential(nn.Linear(4, 8), nn.ReLU(), shared, nn.ReLU(), shared,
                    nn.ReLU(), nn.Linear(8, 1))
net(X)
print(net[2].weight.data[0] == net[4].weight.data[0])
net[2].weight.data[0, 0] = 100
print(net[2].weight.data[0] == net[4].weight.data[0])

tensor([True, True, True, True, True, True, True, True])
tensor([True, True, True, True, True, True, True, True])


# 自定义层

In [36]:
import torch
import torch.nn.functional as F
from torch import nn

class CenteredLayer(nn.Module):
    def __init__(self):
        super().__init__()

    def forward(self, X):
        return X - X.mean()

layer = CenteredLayer()
layer(torch.FloatTensor([1, 2, 3, 4, 5]))

tensor([-2., -1.,  0.,  1.,  2.])

In [37]:
class MyLinear(nn.Module):
    def __init__(self, in_units, units):
        super().__init__()
        self.weight = nn.Parameter(torch.randn(in_units, units))
        self.bias = nn.Parameter(torch.randn(units,))

    def forward(self, X):
        linear = torch.matmul(X, self.weight.data) + self.bias.data
        return F.relu(linear)

linear = MyLinear(5, 3)
linear.weight

Parameter containing:
tensor([[-0.4997, -1.8636, -0.4678],
        [-0.4597, -1.1111, -0.0696],
        [ 2.1552,  0.1946, -1.4019],
        [-0.8776, -0.2580,  0.3035],
        [-0.1760, -0.6681,  0.0667]], requires_grad=True)

# 读写文件

In [38]:
import torch
from torch import nn
from torch.nn import functional as F

x = torch.arange(4)
torch.save(x, 'x-file')

x2 = torch.load('x-file')
x2

tensor([0, 1, 2, 3])

In [39]:
class MLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.hidden = nn.Linear(20, 256)
        self.output = nn.Linear(256, 10)

    def forward(self, x):
        return self.output(F.relu(self.hidden(x)))

net = MLP()
X = torch.randn(size=(2, 20))
Y = net(X)

In [40]:
torch.save(net.state_dict(), 'mlp.params')

In [41]:
clone = MLP()
clone.load_state_dict(torch.load('mlp.params'))
clone.eval()

MLP(
  (hidden): Linear(in_features=20, out_features=256, bias=True)
  (output): Linear(in_features=256, out_features=10, bias=True)
)

In [42]:
import torch
from torch import nn

torch.device('cpu'), torch.cuda.device('cuda'), torch.cuda.device('cuda:1')

(device(type='cpu'),
 <torch.cuda.device at 0x7f2a83543550>)

In [43]:
def try_gpu(i=0):  
    """如果存在，则返回gpu(i)，否则返回cpu()。"""
    if torch.cuda.device_count() >= i + 1:
        return torch.device(f'cuda:{i}')
    return torch.device('cpu')

def try_all_gpus():  
    """返回所有可用的GPU，如果没有GPU，则返回[cpu(),]。"""
    devices = [
        torch.device(f'cuda:{i}') for i in range(torch.cuda.device_count())]
    return devices if devices else [torch.device('cpu')]

try_gpu(), try_gpu(10), try_all_gpus()

(device(type='cpu'), device(type='cpu'), [device(type='cpu')])

In [44]:
X = torch.ones(2, 3, device=try_gpu())
X

tensor([[1., 1., 1.],
        [1., 1., 1.]])

In [45]:
print(X)

tensor([[1., 1., 1.],
        [1., 1., 1.]])


In [46]:
X.device

device(type='cpu')